# Tips
## Deep copy
### Problem definition
Both lists and dicts have methods for creating copies of these objects:

Creating a copy of a list:

In [3]:
list1 = [1, 2, 3]
list2 = list1[:]

list2 is list1

False

Copy of a dict:

In [4]:
dict1 = dict(a=1, b=2, c=3)
dict2 = dict1.copy()

dict2 is dict1

False

However this could be a problem in case your list (or dict) is not just a list, but a complex structure. Both `copy()`s actually copy only the referenced object contents. And because in a complex structure, elements store references to other complex (and usually mutable) objects, the copied object store them as well. This can lead to an undesired behaviour that we usually call **bugs**. Example demonstrating the problem:

In [7]:
crew = [
    dict(firstname='Tomas', lastname='Anderson'),
    dict(firstname='Morpheus'),
    dict(firstname='Trinity'),
    dict(firstname='Cypher'),
]

because `database` is a list, we use `[:]` to get a copy of it. 

In [9]:
replica = crew[:]

Now we may be thinking that `replica` is a full copy of `crew`, we use it to address and modify values:

In [10]:
replica[1]['firstname']

'Morpheus'

We can safely delete or replace items in `replica`.

In [12]:
replica[2] = dict(firstname='Mouse')
del replica[3]

Look, original `crew` is kept untouched:

In [13]:
crew

[{'firstname': 'Tomas', 'lastname': 'Anderson'},
 {'firstname': 'Morpheus'},
 {'firstname': 'Trinity'},
 {'firstname': 'Cypher'}]

However if we attempt to modify a deeper value, like so:

In [16]:
replica[0]['firstname'] = 'Neo'
replica[0]['lastname'] = 'The One'

`crew` appears to be modified, which is probably undesired.

In [17]:
crew

[{'firstname': 'Neo', 'lastname': 'The One'},
 {'firstname': 'Morpheus'},
 {'firstname': 'Trinity'},
 {'firstname': 'Cypher'}]

### The better way: deepcopy

In [19]:
from copy import deepcopy

Defining `crew` once again:

In [23]:
crew = [
    dict(firstname='Tomas', lastname='Anderson'),
    dict(firstname='Morpheus'),
    dict(firstname='Trinity'),
    dict(firstname='Cypher'),
]

In [24]:
replica = deepcopy(crew)

In [25]:
replica[0]['firstname'] = 'Neo'
replica[0]['lastname'] = 'The One'

replica

[{'firstname': 'Neo', 'lastname': 'The One'},
 {'firstname': 'Morpheus'},
 {'firstname': 'Trinity'},
 {'firstname': 'Cypher'}]

`crew` remains unmodified:

In [26]:
crew

[{'firstname': 'Tomas', 'lastname': 'Anderson'},
 {'firstname': 'Morpheus'},
 {'firstname': 'Trinity'},
 {'firstname': 'Cypher'}]

## collections.defaultdict

It's similar to `dict`, but the only difference is that a `defaultdict` will have a default value if that key has not been set yet. In other words, subscript `d['key']` creates an innitializing value and returns it.

> defaultdict(default_factory[, ...]) --> dict with default factory. The default factory is called without arguments to produce a new value when a key is not present

So `default_factory` is a callable (typically a class constructor) or it can be an user defined function that will be called on element access and returned value will be returned.

In [1]:
from collections import defaultdict

So this:

In [2]:
dd = defaultdict(list, a=1, b=2)

#
dd['c'], dict(dd)

([], {'a': 1, 'b': 2, 'c': []})

is equivalent to this:

In [3]:
d = dict(a=1, b=2)
d['c'] = d.get('c', [])

#
d['c'], d

([], {'a': 1, 'b': 2, 'c': []})

When `defaultdict` can be handy (using `list` constructor):

In [4]:
menu = defaultdict(list)

# default_factory=list creates an empty list for 'soups' and 'main courses' first
menu['soups'].append('borsch')
menu['main courses'] += ['steak', 'grilled chicken']

# default_factory is not actually called
menu['desserts'] = ['icecream', 'cake']

dict(menu)

{'desserts': ['icecream', 'cake'],
 'main courses': ['steak', 'grilled chicken'],
 'soups': ['borsch']}

or using `dict` constructor.

In [5]:
menu = defaultdict(dict)

# default_factory=list creates an empty list for 'soups' and 'main courses' first
menu['soups'].update({'borsch': 5})
menu['main courses']['steak'] = 10
menu['main courses']['grilled chicken'] = 8

# default_factory is not actually called
menu['desserts'] = {'icecream': 2, 'cake': 7}

dict(menu)

{'desserts': {'cake': 7, 'icecream': 2},
 'main courses': {'grilled chicken': 8, 'steak': 10},
 'soups': {'borsch': 5}}

## collections.OrderedDict

Keeps the order of insertions.

In [6]:
from collections import OrderedDict

When using usual `dict`, order of inserted elements is not guaranteed:

In [46]:
d = dict()

d[1] = 1
d[2] = 2
d[0] = 0
d

{0: 0, 1: 1, 2: 2}

In [42]:
d.keys()

[0, 1, 2]

`OrderedDict` is a drop-in replacement for `dict` that you can use if the order of inserted elements is important.

In [45]:
od = OrderedDict()

od[1] = 1
od[2] = 2
od[0] = 0

od.keys()

[1, 2, 0]

## collections.Counter

A dict-like structure for keeping things counted.

In [49]:
from collections import Counter

Creating an empty counter:

In [55]:
c = Counter()
c

Counter()

In [56]:
c['cats'] = 10
c

Counter({'cats': 10})

In [57]:
c['dogs'] += 10
c

Counter({'cats': 10, 'dogs': 10})

Or it can be initialized using a dictionary:

In [59]:
website_visitors = Counter({'Latvia': 100, 'Lithuania': 10, 'Estonia': 7})
website_visitors

Counter({'Estonia': 7, 'Latvia': 100, 'Lithuania': 10})

Can also be initialized from a sequence:

In [60]:
letters = Counter('abracadabra')
letters

Counter({'a': 5, 'b': 2, 'c': 1, 'd': 1, 'r': 2})

In [61]:
frequencies = Counter([100, 1, 20, 100, 1, 100])
frequencies

Counter({1: 2, 20: 1, 100: 3})

`Counter` supports addition and subtraction:

In [73]:
firstname_freq = Counter('clerk')
lastname_freq = Counter('maxwell')

firstname_freq + lastname_freq

Counter({'a': 1,
         'c': 1,
         'e': 2,
         'k': 1,
         'l': 3,
         'm': 1,
         'r': 1,
         'w': 1,
         'x': 1})

In [74]:
# elements not found in the firstname are excluded
# elements with negative count are excluded as well
firstname_freq - lastname_freq

Counter({'c': 1, 'k': 1, 'r': 1})

It acts like a dictionary:

In [84]:
letters = Counter('abracadabra')
letters.items()

[('a', 5), ('r', 2), ('b', 2), ('c', 1), ('d', 1)]

It's easy to get total number of all counts:

In [89]:
# it's actually just the len of 'abracadabra'
sum(Counter('abracadabra').values())

11

`Counter.elements()` returns the counted object with elements repeated but in an unpredicted order:

In [80]:
list(Counter('abracadabra').elements())

['a', 'a', 'a', 'a', 'a', 'r', 'r', 'b', 'b', 'c', 'd']

Apparently the most useful method is `Counter.most_common()`:

In [95]:
letters = Counter('abracadabra')

# top 1
letters.most_common(1)

[('a', 5)]